[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gowtham91m/cats-and-dogs-classification/blob/master/cats_and_dogs_classificatoin.ipynb)

In [0]:
%%capture
!pip install kaggle

from zipfile import ZipFile
import io, cv2, fnmatch, shutil, os, getpass, subprocess, random

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import numpy as np
from time import time
from glob import glob
from sklearn.utils import class_weight

In [3]:
os.chdir('/content')
if 'kaggle.txt' not in os.listdir('/content'):
  from google.colab import files
  downloaded = files.upload()
  
os.chdir('/content')
with open('kaggle.txt') as f: key = f.read()
os.environ['KAGGLE_USERNAME']="gowham91m"
os.environ['KAGGLE_KEY']=key
if 'cats_dogs' in os.listdir('/content'):shutil.rmtree('/content/cats_dogs')
os.mkdir('/content/cats_dogs')
os.chdir('/content/cats_dogs')
!kaggle competitions download -c dogs-vs-cats

Saving kaggle.txt to kaggle.txt
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 43.6MB/s]
 92% 250M/271M [00:03<00:00, 51.6MB/s]
100% 271M/271M [00:03<00:00, 75.6MB/s]
 99% 537M/543M [00:08<00:00, 62.5MB/s]
100% 543M/543M [00:08<00:00, 69.8MB/s]


In [4]:
!unzip -q -o train.zip
!unzip -q -o test1.zip

cat_pattern = '*cat.*.jpg'
dog_pattern = '*dog.*.jpg'

images = glob('/content/cats_dogs/train/*.jpg', recursive=True)
cats = fnmatch.filter(images,cat_pattern)
dogs = fnmatch.filter(images,dog_pattern)

os.listdir('/content/cats_dogs')
if 'data' not in os.listdir('/content/cats_dogs'):os.mkdir('/content/cats_dogs/data')
if 'train' not in os.listdir('/content/cats_dogs/data'):os.mkdir('/content/cats_dogs/data/train')
if 'dogs' not in os.listdir('/content/cats_dogs/data/train'):os.mkdir('/content/cats_dogs/data/train/dogs')
if 'cats' not in os.listdir('/content/cats_dogs/data/train'):os.mkdir('/content/cats_dogs/data/train/cats')
  
if 'val' not in os.listdir('/content/cats_dogs/data'):os.mkdir('/content/cats_dogs/data/val')
if 'dogs' not in os.listdir('/content/cats_dogs/data/val'):os.mkdir('/content/cats_dogs/data/val/dogs')
if 'cats' not in os.listdir('/content/cats_dogs/data/val'):os.mkdir('/content/cats_dogs/data/val/cats')

train_dogs_path = '/content/cats_dogs/data/train/dogs'
train_cats_path = '/content/cats_dogs/data/train/cats'

val_dogs_path = '/content/cats_dogs/data/val/dogs'
val_cats_path = '/content/cats_dogs/data/val/cats'

for file in cats: shutil.copy2(file, train_cats_path)
for file in dogs: shutil.copy2(file, train_dogs_path)
  
  
# split train date into train and validation
train_len = len(os.listdir('/content/cats_dogs/data/train/dogs'))
val_len = train_len * 0.3
val_dogs = random.sample(os.listdir(train_dogs_path),int(val_len))
val_cats = random.sample(os.listdir(train_cats_path),int(val_len))


for file in val_dogs:
  try: shutil.move(os.path.join(train_dogs_path,file), val_dogs_path)
  except: pass
for file in val_cats:
  try: shutil.move(os.path.join(train_cats_path,file), val_cats_path)
  except: pass
  
print(len(os.listdir(train_cats_path)))
print(len(os.listdir(val_cats_path)))

print(len(os.listdir(train_dogs_path)))
print(len(os.listdir(val_dogs_path)))

print('total train samples ', len(os.listdir(train_cats_path)) + len(os.listdir(train_dogs_path)))
print('total train samples ', len(os.listdir(val_cats_path)) + len(os.listdir(val_dogs_path)))

8750
3750
8750
3750
total train samples  17500
total train samples  7500


#CNN classifier

In [8]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

batch_size=64
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=( 150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

start_time = time()
model.fit_generator(
        train_generator,
        #steps_per_epoch=18631 // batch_size,
        epochs=32,
        validation_data=validation_generator,
        #validation_steps=10119 // batch_size
        )
model.save_weights('first_try.h5')

print('time taken ',time()-start_time)

Epoch 1/32
274/274 [==============================] - 188s 686ms/step - loss: 0.6740 - acc: 0.5869 - val_loss: 0.6211 - val_acc: 0.6712
Epoch 2/32
274/274 [==============================] - 179s 655ms/step - loss: 0.5808 - acc: 0.6965 - val_loss: 0.5099 - val_acc: 0.7443
Epoch 3/32
274/274 [==============================] - 180s 656ms/step - loss: 0.5189 - acc: 0.7459 - val_loss: 0.4816 - val_acc: 0.7660
Epoch 4/32
274/274 [==============================] - 179s 655ms/step - loss: 0.4636 - acc: 0.7802 - val_loss: 0.4559 - val_acc: 0.7885
Epoch 5/32
274/274 [==============================] - 180s 657ms/step - loss: 0.4238 - acc: 0.8046 - val_loss: 0.4466 - val_acc: 0.8041
Epoch 6/32
274/274 [==============================] - 180s 656ms/step - loss: 0.3948 - acc: 0.8210 - val_loss: 0.3794 - val_acc: 0.8220
Epoch 7/32
274/274 [==============================] - 180s 656ms/step - loss: 0.3596 - acc: 0.8415 - val_loss: 0.3292 - val_acc: 0.8616
Epoch 8/32
274/274 [============================

#Transfer learning

In [32]:
BASE_MODEL = 'VGG16'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)
val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

vgg_model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    
vgg_model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
vgg_model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)])
vgg_model.save_weights('VGG16.h5')

print('time taken ',time()-start_time)

Epoch 1/16
274/274 [==============================] - 326s 1s/step - loss: 0.4903 - acc: 0.7797 - val_loss: 0.3391 - val_acc: 0.8727
Epoch 2/16
274/274 [==============================] - 311s 1s/step - loss: 0.3377 - acc: 0.8726 - val_loss: 0.2705 - val_acc: 0.8969
Epoch 3/16
274/274 [==============================] - 313s 1s/step - loss: 0.2928 - acc: 0.8849 - val_loss: 0.2416 - val_acc: 0.9075
Epoch 4/16
274/274 [==============================] - 315s 1s/step - loss: 0.2664 - acc: 0.8961 - val_loss: 0.2263 - val_acc: 0.9132
Epoch 5/16
274/274 [==============================] - 314s 1s/step - loss: 0.2507 - acc: 0.9002 - val_loss: 0.2202 - val_acc: 0.9147
Epoch 6/16
274/274 [==============================] - 315s 1s/step - loss: 0.2404 - acc: 0.9034 - val_loss: 0.1999 - val_acc: 0.9220
Epoch 7/16
274/274 [==============================] - 314s 1s/step - loss: 0.2319 - acc: 0.9062 - val_loss: 0.1922 - val_acc: 0.9260
Epoch 8/16
274/274 [==============================] - 315s 1s/step - 

In [16]:
BASE_MODEL = 'vgg19'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('vgg19-transferlearning.model', monitor='val_acc', save_best_only=True)])
model.save_weights('vgg19.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
Epoch 1/16
547/547 [==============================] - 370s 676ms/step - loss: 0.4825 - acc: 0.7809 - val_loss: 0.3768 - val_acc: 0.8321
Epoch 2/16
547/547 [==============================] - 355s 650ms/step - loss: 0.3353 - acc: 0.8632 - val_loss: 0.2976 - val_acc: 0.8768
Epoch 3/16
547/547 [==============================] - 358s 655ms/step - loss: 0.2954 - acc: 0.8792 - val_loss: 0.2573 - val_acc: 0.8976
Epoch 4/16
547/547 [==============================] - 357s 653ms/step - loss: 0.2747 - acc: 0.8856 - val_loss: 0.2554 - val_acc: 0.8881
Epoch 5/16
547/547 [==============================] - 355s 648ms/step - loss: 0.2577 - acc: 0.8926 - val_loss: 0.2327 - val_acc: 0.9069
Epoch 6/16
547/547 [==============================] - 355s 648ms/step - loss: 0.2473 - acc: 0.8987 - val_loss: 0.2223 - val_acc: 0.9065
Epoch 7/16
547/547 [==============================] - 356s 651ms/step - loss: 0.2362 - acc: 0.9054 

In [14]:
BASE_MODEL = 'ResNet50'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('ResNet50-transferlearning.model', monitor='val_acc', save_best_only=True)])
model.save_weights('ResNet50.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
Epoch 1/16
547/547 [==============================] - 320s 585ms/step - loss: 0.3268 - acc: 0.8855 - val_loss: 0.1813 - val_acc: 0.9453
Epoch 2/16
547/547 [==============================] - 317s 580ms/step - loss: 0.1606 - acc: 0.9486 - val_loss: 0.1292 - val_acc: 0.9579
Epoch 3/16
547/547 [==============================] - 316s 577ms/step - loss: 0.1262 - acc: 0.9563 - val_loss: 0.1124 - val_acc: 0.9603
Epoch 4/16
547/547 [==============================] - 323s 590ms/step - loss: 0.1092 - acc: 0.9616 - val_loss: 0.0993 - val_acc: 0.9648
Epoch 5/16
547/547 [==============================] - 316s 577ms/step - loss: 0.1042 - acc: 0.9617 - val_loss: 0.0907 - val_acc: 0.9653
Epoch 6/16
547/547 [==============================] - 321s 587ms/step - loss: 0.0989 - acc: 0.9637 - val_loss: 0.0899 - val_acc: 0.9669
Epoch 7/16
547/547 [==============================] - 317s 579ms/step - loss: 0.0946 - acc: 0.9646 

In [17]:
BASE_MODEL = 'InceptionV3'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('InceptionV3-transferlearning.model', monitor='val_acc', save_best_only=True)])
model.save_weights('InceptionV3.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
87916544/87910968 [==============================] - 3s 0us/step
Epoch 1/16
547/547 [==============================] - 290s 530ms/step - loss: 0.3235 - acc: 0.8516 - val_loss: 0.2213 - val_acc: 0.9057
Epoch 2/16
547/547 [==============================] - 281s 513ms/step - loss: 0.2277 - acc: 0.9031 - val_loss: 0.2134 - val_acc: 0.9081
Epoch 3/16
547/547 [==============================] - 284s 519ms/step - loss: 0.2046 - acc: 0.9133 - val_loss: 0.2145 - val_acc: 0.9087
Epoch 4/16
547/547 [==============================] - 282s 516ms/step - loss: 0.1987 - acc: 0.9170 - val_loss: 0.1949 - val_acc: 0.9192
Epoch 5/16
547/547 [==============================] - 281s 514ms/step - loss: 0.1874 - acc: 0.9198 - val_loss: 0.1916 - val_acc: 0.9215
Epoch 6/16
547/547 [==============================] - 281s 513ms/step - loss: 0.1837 - acc: 0.9210 - val_loss: 0.1814 - val_acc: 0.9247
Epoch 7/16
547/547 [==============

In [18]:
BASE_MODEL = 'Xception'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)])
model.save_weights('Xception.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
83689472/83683744 [==============================] - 2s 0us/step
Epoch 1/16
547/547 [==============================] - 352s 643ms/step - loss: 0.1512 - acc: 0.9373 - val_loss: 0.0958 - val_acc: 0.9623
Epoch 2/16
547/547 [==============================] - 341s 624ms/step - loss: 0.1019 - acc: 0.9588 - val_loss: 0.0990 - val_acc: 0.9604
Epoch 3/16
547/547 [==============================] - 342s 624ms/step - loss: 0.0968 - acc: 0.9617 - val_loss: 0.0896 - val_acc: 0.9628
Epoch 4/16
547/547 [==============================] - 340s 622ms/step - loss: 0.0875 - acc: 0.9656 - val_loss: 0.0835 - val_acc: 0.9672
Epoch 5/16
547/547 [==============================] - 341s 623ms/step - loss: 0.0867 - acc: 0.9656 - val_loss: 0.0862 - val_acc: 0.9651
Epoch 6/16
547/547 [==============================] - 343s 627ms/step - loss: 0.0861 - acc: 0.9660 - val_loss: 0.0841 - val_acc: 0.9669
Epoch 7/16
547/547 [==============

In [24]:
BASE_MODEL = 'DenseNet169'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(250, 250),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(250, 250),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 250, 250, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)])
model.save_weights('Xception.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
53182464/53178568 [==============================] - 1s 0us/step
Epoch 1/16
547/547 [==============================] - 476s 870ms/step - loss: 0.0731 - acc: 0.9710 - val_loss: 0.0445 - val_acc: 0.9824
Epoch 2/16
547/547 [==============================] - 456s 834ms/step - loss: 0.0432 - acc: 0.9833 - val_loss: 0.0483 - val_acc: 0.9823
Epoch 3/16
547/547 [==============================] - 458s 837ms/step - loss: 0.0398 - acc: 0.9850 - val_loss: 0.0430 - val_acc: 0.9836
Epoch 4/16
547/547 [==============================] - 460s 840ms/step - loss: 0.0308 - acc: 0.9883 - val_loss: 0.0415 - val_acc: 0.9836
Epoch 5/16
547/547 [==============================] - 456s 833ms/step - loss: 0.0275 - acc: 0.9900 - val_loss: 0.0396 - val_acc: 0.9845
Epoch 6/16
547/547 [==============================] - 456s 833ms/step - loss: 0.0230 - acc: 0.9909 - val_loss: 0.0417 - val_acc: 0.9844
Epoch 7/16
547/547 [==============

In [5]:
BASE_MODEL = 'DenseNet121'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(250, 250),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(250, 250),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 250, 250, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)])
model.save_weights('Xception.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
30015488/30011760 [==============================] - 3s 0us/step
Epoch 1/16
547/547 [==============================] - 421s 770ms/step - loss: 0.0930 - acc: 0.9625 - val_loss: 0.0699 - val_acc: 0.9745
Epoch 2/16
547/547 [==============================] - 407s 743ms/step - loss: 0.0505 - acc: 0.9814 - val_loss: 0.0599 - val_acc: 0.9783
Epoch 3/16
547/547 [==============================] - 410s 750ms/step - loss: 0.0420 - acc: 0.9834 - val_loss: 0.0575 - val_acc: 0.9800
Epoch 4/16
547/547 [==============================] - 409s 748ms/step - loss: 0.0380 - acc: 0.9854 - val_loss: 0.0542 - val_acc: 0.9787
Epoch 5/16
547/547 [==============================] - 421s 770ms/step - loss: 0.0327 - acc: 0.9870 - val_loss: 0.0583 - val_acc: 0.9781
Epoch 6/16
547/547 [==============================] - 421s 769ms/step - loss: 0.0322 - acc: 0.9870 - val_loss: 0.0514 - val_acc: 0.9817
Epoch 7/16
547/547 [==============

In [0]:
# running on TPU

# !pip install keras==2.1
# import tensorflow as tf
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense


BASE_MODEL = 'Xception'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(loss='binary_crossentropy'
              ,optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator
        #class_weight = class_weights,
        ,callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)]
        )
model.save_weights('Xception.h5')

print('time taken ',time()-start_time)